In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
import double_integrator_utils
import robust_value_approx.value_to_optimization as value_to_optimization
import robust_value_approx.hybrid_linear_system as hybrid_linear_system
import robust_value_approx.model_bounds as model_bounds
import robust_value_approx.gurobi_torch_mip as gurobi_torch_mip

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import copy
import gurobipy

In [ ]:
# generating data
vf = double_integrator_utils.get_value_function()
x0_lo = -1. * torch.ones(vf.sys.x_dim, dtype=vf.dtype)
x0_up = 1. * torch.ones(vf.sys.x_dim, dtype=vf.dtype)
num_breaks = [100] * vf.sys.x_dim

In [ ]:
x_samples_train, v_samples_train = vf.get_value_sample_grid(x0_lo, x0_up, num_breaks, update_progress=True)

In [ ]:
x_samples_file = '../data/robust_value_demo_x'
v_samples_file = '../data/robust_value_demo_v'

In [ ]:
torch.save(x_samples_train, x_samples_file + '_train.pt')
torch.save(v_samples_train, v_samples_file + '_train.pt')

In [ ]:
x_samples_train = torch.load(x_samples_file + '_train.pt')
v_samples_train = torch.load(v_samples_file + '_train.pt')

In [ ]:
batch_size = 100
nn_width = 16
nn_depth = 1
learning_rate = 1e-3
momentum = .25
train_data_set = torch.utils.data.TensorDataset(x_samples_train,
                                                v_samples_train)
train_data_loader = torch.utils.data.DataLoader(train_data_set,
                                                batch_size=batch_size,
                                                shuffle=True)
nn_layers = [nn.Linear(vf.sys.x_dim, nn_width), nn.ReLU()]
for i in range(nn_depth):
    nn_layers += [nn.Linear(nn_width, nn_width), nn.ReLU()]
nn_layers += [nn.Linear(nn_width, 1)]
model = nn.Sequential(*nn_layers).double()
l2_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(),
                            lr=learning_rate, momentum=momentum)
robust_model = copy.deepcopy(model)
robust_l2_fn = torch.nn.MSELoss(reduction="mean")
robust_optimizer = torch.optim.SGD(robust_model.parameters(),
                                   lr=learning_rate, momentum=momentum)
mb = model_bounds.ModelBounds(robust_model, vf)
writer = SummaryWriter()
num_total_iter_done = 0

In [ ]:
num_l2_iter_desired = 100
num_robust_iter_desired = 100
num_l2_iter_done = 0
num_robust_iter_done = 0
robust_weight = .5
while True:
    if num_l2_iter_done >= num_l2_iter_desired and num_robust_iter_done >= num_robust_iter_desired:
        break
    for batch_data, batch_label in train_data_loader:
        # nonrobust (pure l2 regression)
        y_pred = model(batch_data)
        l2 = l2_fn(y_pred, batch_label)
        if num_l2_iter_done < num_l2_iter_desired:
            epsilon = torch.Tensor([0.]).type(vf.dtype)
        else:
            (Q1, Q2, q1, q2, k,
             G1, G2, h,
             A1, A2, b) = mb.upper_bound_opt(model, x0_lo, x0_up)
            mb_prob = gurobi_torch_mip.GurobiTorchMIQP(vf.dtype)
            mb_prob.gurobi_model.setParam(gurobipy.GRB.Param.OutputFlag, False)
            # the continuous variables: y = [x, s, z]
            y = mb_prob.addVars(Q1.shape[0], lb=-gurobipy.GRB.INFINITY,
                                vtype=gurobipy.GRB.CONTINUOUS, name="y")
            # the binary variable: γ = [α, β]
            gamma = mb_prob.addVars(Q2.shape[0], vtype=gurobipy.GRB.BINARY, name="gamma")
            mb_prob.setObjective([.5 * Q1, .5 * Q2],
                                 [(y, y), (gamma, gamma)],
                                 [q1, q2], [y, gamma], k,
                                 gurobipy.GRB.MINIMIZE)
            for i in range(G1.shape[0]):
                mb_prob.addLConstr([G1[i, :], G2[i, :]], [y, gamma],
                                   gurobipy.GRB.LESS_EQUAL, h[i])
            for i in range(A1.shape[0]):
                mb_prob.addLConstr([A1[i, :], A2[i, :]], [y, gamma],
                                   gurobipy.GRB.EQUAL, b[i])
            mb_prob.gurobi_model.update()
            mb_prob.gurobi_model.optimize()
            epsilon = mb_prob.compute_objective_from_mip_data_and_solution(penalty=1e-8)
        loss = l2 + 0.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # robust regression
        y_pred_robust = robust_model(batch_data)
        robust_l2 = robust_l2_fn(y_pred_robust, batch_label)
        if num_l2_iter_done < num_l2_iter_desired:
            robust_epsilon = torch.Tensor([0.]).type(vf.dtype)
            num_l2_iter_done += 1
        else:
            (Q1, Q2, q1, q2, k,
             G1, G2, h,
             A1, A2, b) = mb.upper_bound_opt(robust_model, x0_lo, x0_up)
            mb_prob = gurobi_torch_mip.GurobiTorchMIQP(vf.dtype)
            mb_prob.gurobi_model.setParam(gurobipy.GRB.Param.OutputFlag, False)
            y = mb_prob.addVars(Q1.shape[0], lb=-gurobipy.GRB.INFINITY,
                                vtype=gurobipy.GRB.CONTINUOUS, name="y")
            gamma = mb_prob.addVars(Q2.shape[0], vtype=gurobipy.GRB.BINARY, name="gamma")
            mb_prob.setObjective([.5 * Q1, .5 * Q2],
                                 [(y, y), (gamma, gamma)],
                                 [q1, q2], [y, gamma], k,
                                 gurobipy.GRB.MINIMIZE)
            for i in range(G1.shape[0]):
                mb_prob.addLConstr([G1[i, :], G2[i, :]], [y, gamma],
                                   gurobipy.GRB.LESS_EQUAL, h[i])
            for i in range(A1.shape[0]):
                mb_prob.addLConstr([A1[i, :], A2[i, :]], [y, gamma],
                                   gurobipy.GRB.EQUAL, b[i])
            mb_prob.gurobi_model.update()
            mb_prob.gurobi_model.optimize()
            robust_epsilon = mb_prob.compute_objective_from_mip_data_and_solution(penalty=1e-8)
            num_robust_iter_done += 1
        robust_loss = robust_l2 + robust_weight*torch.exp(-robust_epsilon)
        robust_optimizer.zero_grad()
        robust_loss.backward()
        robust_optimizer.step()
        # logging
        writer.add_scalars('L2',
                           {'l2 training': l2.item(),
                            'robust training': robust_l2.item()},
                           num_total_iter_done)
        if num_l2_iter_done >= num_l2_iter_desired:
            writer.add_scalars('Epsilon',
                               {'l2 training': torch.clamp(-epsilon, 0.).item(),
                                'robust training': torch.clamp(-robust_epsilon, 0.).item()},
                               num_total_iter_done)
        num_total_iter_done += 1

In [ ]:
torch.save(model, '../data/robust_value_demo_model.pt')
torch.save(robust_model, '../data/robust_value_demo_robust_model.pt')